# Day 20 - Qwen2.5 72b

In [1]:
from collections import deque

def parse_input(filename):
    with open(filename, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    return grid

def find_start_end(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)
    return start, end

def is_valid_move(grid, x, y):
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])

def bfs(grid, start, end):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    queue = deque([(start, 0)])
    visited = set([start])
    distances = {start: 0}
    
    while queue:
        (x, y), dist = queue.popleft()
        if (x, y) == end:
            return distances
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(grid, nx, ny) and (nx, ny) not in visited and grid[nx][ny] == '.' or:
                queue.append(((nx, ny), dist + 1))
                visited.add((nx, ny))
                distances[(nx, ny)] = dist + 1
    return distances

def find_cheats(grid, start, end, distances):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    cheats = []
    
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            if grid[x][y] == '#' and (x, y) not in distances:
                continue
            for dx1, dy1 in directions:
                for dx2, dy2 in directions:
                    nx1, ny1 = x + dx1, y + dy1
                    nx2, ny2 = nx1 + dx2, ny1 + dy2
                    if is_valid_move(grid, nx1, ny1) and is_valid_move(grid, nx2, ny2):
                        if grid[nx1][ny1] == '#' and grid[nx2][ny2] == '#':
                            if (nx2, ny2) in distances:
                                time_saved = distances[(nx2, ny2)] - distances[(x, y)] - 2
                                if time_saved >= 100:
                                    cheats.append((x, y, nx2, ny2, time_saved))
    return cheats

def main():
    grid = parse_input('input.txt')
    start, end = find_start_end(grid)
    distances = bfs(grid, start, end)
    cheats = find_cheats(grid, start, end, distances)
    print(f"Number of cheats that save at least 100 picoseconds: {len(cheats)}")

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (238562234.py, line 32)

In [2]:
import sys
from collections import deque

def read_grid(filename):
    with open(filename, 'r') as f:
        grid = [line.strip() for line in f.readlines() if line.strip()]
    return grid

def find_positions(grid, target):
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            if grid[y][x] == target:
                return x, y
    return None

def bfs(grid, start, walls):
    rows, cols = len(grid), len(grid[0])
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    d = [[-1 for _ in range(cols)] for _ in range(rows)]
    queue = deque()
    queue.append(start)
    visited[start[1]][start[0]] = True
    d[start[1]][start[0]] = 0
    while queue:
        x, y = queue.popleft()
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < cols and 0 <= ny < rows:
                if not visited[ny][nx] and grid[ny][nx] != '#' and (ny, nx) not in walls:
                    visited[ny][nx] = True
                    d[ny][nx] = d[y][x] + 1
                    queue.append((nx, ny))
    return d

def main():
    grid = read_grid('input.txt')
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    start = find_positions(grid, 'S')
    end = find_positions(grid, 'E')

    d_S = bfs(grid, start, set())
    d_E = bfs(grid, end, set())

    T = d_S[end[1]][end[0]]
    print(f"Fastest time without cheating: {T} picoseconds")

    cheat_saves = []
    walls = set()
    for y in range(rows):
        for x in range(cols):
            if grid[y][x] == '#':
                walls.add((y, x))

    from collections import deque
    queue = deque()
    queue.append((*start, 0, False, 2))  # x, y, time, cheat_used, cheat_remaining
    visited = set()
    visited.add((start[0], start[1], False, 2))
    min_T_prime = float('inf')

    while queue:
        x, y, t, cheat_used, cheat_remaining = queue.popleft()
        if (x, y) == end:
            min_T_prime = min(min_T_prime, t)
            continue
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < cols and 0 <= ny < rows:
                cell = grid[ny][nx]
                if cheat_used:
                    if cell == '#' and cheat_remaining >= 1:
                        if (nx, ny, True, cheat_remaining - 1) not in visited:
                            visited.add((nx, ny, True, cheat_remaining - 1))
                            queue.append((nx, ny, t + 1, True, cheat_remaining - 1))
                    elif cell != '#':
                        if (nx, ny, True, cheat_remaining - 1) not in visited:
                            visited.add((nx, ny, True, cheat_remaining - 1))
                            queue.append((nx, ny, t + 1, True, cheat_remaining - 1))
                else:
                    if cell == '.':
                        if (nx, ny, False, 2) not in visited:
                            visited.add((nx, ny, False, 2))
                            queue.append((nx, ny, t + 1, False, 2))
                    if cell == '#':
                        if (nx, ny, True, 1) not in visited:
                            visited.add((nx, ny, True, 1))
                            queue.append((nx, ny, t + 1, True, 1))

    if min_T_prime != float('inf'):
        time_saved = T - min_T_prime
        print(f"Minimal time with cheating: {min_T_prime} picoseconds")
        print(f"Time saved: {time_saved} picoseconds")
        if time_saved >= 100:
            print("At least one cheat saves at least 100 picoseconds.")
        else:
            print("No cheat saves at least 100 picoseconds.")
    else:
        print("No path found with cheating.")

if __name__ == "__main__":
    main()

Fastest time without cheating: 9484 picoseconds


KeyboardInterrupt: 

In [ ]:
from collections import deque

def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f]
    return lines

def find_positions(racetrack):
    start = None
    end = None
    for y, row in enumerate(racetrack):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
    return start, end

def is_valid(x, y, racetrack):
    return 0 <= x < len(racetrack[0]) and 0 <= y < len(racetrack)

def bfs_shortest_path(racetrack, start, end):
    queue = deque([(start, 0)])
    visited = set([start])
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    while queue:
        (x, y), steps = queue.popleft()
        if (x, y) == end:
            return steps
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny, racetrack) and (nx, ny) not in visited and racetrack[ny][nx] != '#':
                visited.add((nx, ny))
                queue.append(((nx, ny), steps + 1))
    return float('inf')

def bfs_cheating_paths(racetrack, start, end, normal_path_length):
    cheats = []
    queue = deque([(start, 0, None, None)])
    visited = set([(start, None, None)])
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    while queue:
        (x, y), steps, cheat_start, cheat_end = queue.popleft()
        if (x, y) == end:
            if cheat_start is not None and cheat_end is not None:
                time_saved = normal_path_length - steps
                if time_saved >= 100:
                    cheats.append(((cheat_start, cheat_end), time_saved))
        
 continue:
        for dx, dy) in directions:
        for (x, y) in end:
        if (x, y) == end:
            if (x, 1):
            else):
                if normal_path_length + 1):
            else) and steps + 1,            else):
            else):
                if normal_path_length + 1)]
            for dx, dy) in visited((nx, y) == float)
 and cheat,):
                    else, None, None, None)
 in visited() and visited((x, y), == 0:
            else) not None:
 None) and steps + 1)]
            for dx, dy) in visited([nx, y) == 0)
 and cheat_start = None, and visited((dx, y) == 1)]
            for dx, in normal((0, y) == 1))
            cheat_end = (nx, y) == 1)]            visited = None) and steps + 1)
 in visited((0, y) == 1)]
            for dx, dy) in visited((0, y) == 1)]            visited = None) and steps + 1)]
            if cheat_start = None, None) and steps + 1)]
            visited[(0, y) == 1)]
            cheat_end = (nx, y) == 0)]            visited = None) and steps + 1)]
            if cheat_start = None, None) and steps + 1)]
            visited = None)
 and steps + 1)]            visited = None) and steps + 1)]
            if time_saved == 0) and 1)]            visited = None) and steps + 1)]
            for dx, dy) in visited((0, y) == 1)]
            else):
 None
            
)
            cheat:
 None)
            else) visited)
0)2           )
            for dx, dy in directions:
0:
8        else):
            else):
            if end, None):
 None)]            queue) rac)
            else:
 None):
                if end, y)
10)
        if (x, y) == 0) 1)
            return (x)            None, None)
 None)]
            for dx,            None, None) and steps()[0] in end, y). 0, y) == 0)]
            for dx) in normal)]
            if dx)            None, None) and steps + 1)
            if)
        (x, y) == 1)]
            for dx)
 end) in 1, y) == 0)]
            None):
            for dx,            None) and steps + 1)]
            for dx) rac_path.
)
)]
):
            if (x, y) == 1)]
            for dx, None):
                None) and steps + 1)])
            if (x, y) == 0)]
            for dx, dy in directions):
[(0, y) == 1)]            (0, y) == 1)]            visited)
 and steps + 1)]
            for dx, dy in directions)][(0, -1)]            visited = None) and steps + 1)]            visited() and steps + 1)]
            for dx, y) == 0)]
            for dx, y) directions.
 None) and steps + 1)]
            if (x, y) == 0)]            visited() and steps + 1)]:
):
            None, None) and steps + 1)]
            visited() and steps + 1)] None)
 and steps + 1)]            visited = None) and steps + 1)]
            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            visited = None) and steps + 1)]
            for dx, dy in directions)][(0, y) == 1)]
            visited() and steps + 1)] None)
 and steps + 1)]            visited.add None) and steps + 1)]
            for dx, dy in directions)][(0, 1)]
            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            for dx, 1)]
            visited = None):
 None) and steps + 1)]            visited = None) and steps + 1)]
            for dx, y) == 1)]            visited.add None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited() and steps + 1)]
            for dx, dy in directions)][(0, y) == 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited() and steps + 1)]
            visited = None)]
            for dx, y) None)]
            for dx, 1)]
            if (x, 1)]
            for dx, dy in directions)][(0, y) == 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            visited = None)]
            for dx, dy in directions)][(0, -1)]            visited = None) and steps + 1)]            visited = None)]:
            for dx, y) == 1)])
            visited = None)]            visited) None)]            for dx, dy) in directions)][(0, -1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            visited = None) and steps + 1)]
            visited = None) and steps + 1)]
            visited = None) and steps + 1)]            visited) None):
, None):
            for 0)]            visited = None)]
            visited = None)]            visited = None) and steps + 1)]
            visited = None)]            visited = None):
            visited = None) and steps + 1)]
            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None)]            visited = None) and steps + 1)]
            visited = None)]            visited = None)] and steps + 1)]
            visited = None) and steps + 1)]            visited = None)
            for dx, dy) in visited) and steps + 1)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]            visited = None)]            visited = None)]            visited = None) and steps + 1)]            visited = None)]            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            visited = None)]            visited = None) and steps + 1)]
            visited = None)]            visited = None) and steps + 1)]
            visited = None) and steps() 1)]
            visited = None) and steps + 1)]            visited = None) and steps + 1)]
            visited = None)]            visited = None)]            visited = None) and steps + 1)]            visited = None) and steps + )]            visited.add None)]:
)
       ):
:
):
,1)
    normal_path_length, ) rac_path.
)
        (dx, y))
            visited = None)]            visited = None)]            visited = None):
            visited = None)]            visited = None)]            visited in directions)]            visited in None) and steps + )]
            visited(( None)]            visited in None)]:
):
            visited[(1, 1)]            visited in None)]            visited in None) and steps + 0)]            visited in None) and steps + 0)]
            visited = None)]            visited in None)]            visited in None)]            visited = None)]            visited in None) and steps + 1)]            visited in None)] and steps + 0)]            visited in None)] and steps + 1)]:
            cheating_path(rac_path):
    directions =]:
    cheating = None) and steps + 0)]            visited = None) and steps + 0)]
            visited = None)]:
    for dx, None, and personal) None)]            visited = None)] and steps + 1)]
            visited(( None)]            visited in None)]:
            visited in None)]:
    directions = [(0, 0)]            visited in directions)]:
    for dx, y) and steps + 1)]
            visited(( None)]            steps + 1)]
:
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited = None)]:
    for dx, dy) None, and steps + )]            visited.add(None) and steps + 1)]
            visited(( None)]            for dx, dy in directions:
    for dx, dy) None, and steps + 1,5):
            visited.add None)]:
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited(( None)]            steps + 1)]
            visited.add None)]?
    for dx, dy) None, and steps + 1)]            visited.add None) and steps + 1)]
            visited(( None)] None, None steps + 0)]
            visited = None)]            steps + 1)]
            visited.add None)]?
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited = None)]?
    for dx, dy) None, and steps + )]            visited.add None) and steps + 1)]
            visited.add None)]?
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited.add None)]?
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited.add None)]?
    for dx, dy) None, and steps + 0)]            visited.add None) and steps + 1)]
            visited.add None)]?
    for dx, dy)

## Part 2